In [1]:
import pandas as pd
import numpy as np
import openpyxl as opxl
import os.path
import pycountry
import pycountry_convert as pc
import datetime as dt
import json
import os.path

def importExcelSheets():
    magento = pd.read_excel('database_sheets/magento_may.xlsx')
    new_magento = pd.read_excel('database_sheets/magento_july.xlsx')
    magento_sept = pd.read_excel('database_sheets/magento_sept.xlsx')
    lot_master = pd.read_excel('database_sheets/lot_master.xlsx', dtype = object)
    prms = pd.read_excel('database_sheets/prms.xlsx')
    unspsc_codes = pd.read_excel('database_sheets/unspsc_codes.xlsx')
    unspsc_codes.columns = unspsc_codes.iloc[0]
    origin = pd.read_excel('database_sheets/country_of_origin.xlsx', dtype=object)
    pd.set_option("max_rows", None)
    pd.set_option("max_columns", None)
    return [magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]

def tidyDescription(desc):
    desc = desc.replace('&TRADE', '')
    desc = desc.replace('Â®', '')
    desc = desc.replace('Î¼', '')
    desc = desc.replace('& Growâ„¢', '')
    desc = desc.replace('â‰', '')
    desc = desc.replace('&Beta', '')
    desc = desc.replace('Â‰¥', '')
    desc = desc.replace('Â', '')
    desc = desc.replace('<em>', '')
    desc = desc.replace('</em>', '')
    desc = desc.replace('®','')
    desc = desc.replace('<F128>','')
    desc = desc.replace('<130>','')
    desc = desc.replace('<F255>','')
    desc = desc.replace('&deg;', '°')
    desc = desc.replace('deg ', '°')
    if desc.endswith(','):
        desc = desc[:len(desc)-1]
    return desc

database_sheets = importExcelSheets()

In [4]:
revision = pd.read_excel('forms/GlobalProductRevisionFile (New).xlsx')
revision.columns = revision.iloc[7]
new_columns = [i.strip() if type(i) == str else i for i in revision.columns]
revision.columns = new_columns
revision

,Group Key,vwr.com URL,Action for this item,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Search Keywords,NaN,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Text related information,NaN,Energy Efficient,Water Efficient,Waste Reducing,Sustainable Materials,Low Manufacturing Impact,Safer Human and Environmental Health,Sustainable Packaging,Product Transparency and Disclosure,Acceptable Verification Method,Additional Product Data/Information
0,-You must provide the URL requested in Colu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"-Other than Columns A and B, all fields are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To the left is part number specific informatio...,NaN,NaN,NaN,To the right is the parent/product information...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-Add new columns for each Order Tab and Spe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-When adding a Cat. No. to an existing page...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A unique value to group part numbers together ...,Address of existing webpage,"Choose from add item, remove item, or change i...","Products(s) to be added, removed, or altered \...",NaN,NaN,NaN,Indicate 'Y' value for if the SKU is an Access...,If needed:\nAdd a sub-heading to break part nu...,Add Attribute Names to Row 9; be specific abou...,NaN,NaN,NaN,NaN,NaN,Add terms here that customers will use to find...,NaN,Provide information only if a change is needed.,NaN,NaN,NaN,NaN,If this is an Environmentally Preferable Produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please explain your EPP claim below (ie: speci...,Please explain your EPP claim below (ie: speci...
7,Group Key,vwr.com URL,Action for this item,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Search Keywords,NaN,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Text related information,NaN,Energy Efficient,Water Efficient,Waste Reducing,Sustainable Materials,Low Manufacturing Impact,Safer Human and Environmental Health,Sustainable Packaging,Product Transparency and Disclosure,Acceptable Verification Method,Additional Product Data/Information
8,Examples below…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ex. Description,ex. Color,ex. Capacity,ex. Electrical,NaN,NaN,NaN,NaN,*NOTE: Parent/Product information only needs t...,NaN,NaN,NaN,NaN,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,"ex. ENERGY STAR, FSC, Ecologo, etc.",Include EPP Narrative
9,1,https://us.vwr.com/store/catalog/product.jsp?c...,Add item,11111-111,123-4567,CA11111-111,AA-5ML,NaN,NaN,Wide-Mouth Beakers,White,NaN,NaN,NaN,NaN,VWR Beaker; 5mL Beaker; Standard-Grade Beakers,NaN,VWR®,Best Glassware Company,Standard-Grade Beakers,"Change within the bullet point \n""With spout"" ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def fillGlobalProductRevision(filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept):
    revision = pd.read_excel('forms/GlobalProductRevisionFile (New).xlsx')
    revision.columns = revision.iloc[7]
    new_columns = [i.strip() if type(i) == str else i for i in revision.columns]
    revision.columns = new_columns
    wb = opxl.load_workbook(filename)
    skus = wb.active
    for i in range(2, skus.max_row+1):
        revision.loc[i+10, 'Supplier Part No.'] = str(skus['A'+str(i)].value)

    for i in range(12, len(revision)):
        sku = revision.loc[i, 'Supplier Part No.']
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]

        if not product_info_sept.empty:
            keywords = product_info_sept['keywords'].values[0]
            name = product_info_sept['name'].values[0]
            description = product_info_sept['description'].values[0]

            if type(keywords) == str:
                keywords = keywords.replace(', ', ';')
                keywords = keywords.replace(',', ';')
                keywords = keywords.replace(' |', ';')
                keywords = keywords.replace('| ', ';')
                keywords = keywords.replace(' | ', ';')
                keywords = keywords.replace('|', ';')
                revision.loc[i, 'Search Keywords'] = keywords
            
            revision.loc[i, 'Supplier Name'] = 'MP Biomedicals'

            if type(name) == str:
                revision.loc[i, 'Product Title\n(max. 100 characters)'] = tidyDescription(name)
            
            if type(description) == str:
                revision.loc[i, 'Text related information'] = tidyDescription(description)
                               
    new_revision = opxl.load_workbook('forms/GlobalProductrevisionFile (New).xlsx')
    sheet = new_revision.active
    if sheet.max_row < len(revision):
        sheet.append([''])
        for j in range(len(revision) - sheet.max_row + 4):
            sheet.insert_rows(sheet.max_row)
    i = 12
    for row in sheet.iter_rows(min_row=14):
        if i < (len(revision)):
            for j in range(len(revision.columns)):
                row[j].value = revision.iloc[i, j]
        else:
            break
        i += 1
            
    new_revision.save('outputs/revision_outputs/GlobalProductRevision_output.xlsx')

In [21]:
fillGlobalProductRevision('input_sheets/VWR testing.xlsx', database_sheets[0], database_sheets[1], database_sheets[2], database_sheets[3], database_sheets[4], database_sheets[5], database_sheets[6])

In [23]:
revision = pd.read_excel('forms/GlobalProductRevisionFile_Chemicals (New).xlsx')
revision.columns = revision.iloc[8]
new_columns = [i.strip() if type(i) == str else i for i in revision.columns]
revision.columns = new_columns
revision

,Group Key,vwr.com URL,Action Requested,VWR-NA Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,NaN,Size,Pack Type,Search Keywords,NaN,Product Name,Product Text,Key Features/Benefits,NaN,CAS,MDL,EINECS,REACH,Merck Index,UN,Hazard Class,Packing Group,GHS Symbol(s),GHS Signal Word,Density,Boiling Point,Melting Point,Flash Point,Storage Temperature,NaN,Test 1,Result 1,Test 2,Result 2,Test 3,Result 3,Test 4,Result 4,Test 5,Result 5
0,"-To submit a Revision File, you must alread...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"-On this form, only populate fields that re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-Use Column C to describe the change you ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-If you leave a column empty than any exist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Product Characteristics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Product Specifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Part Numbers and Grouping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Item Descriptions,NaN,NaN,NaN,Text Fields,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Add additional columns as needed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Identification,NaN,NaN,NaN,NaN,Hazard,NaN,NaN,NaN,NaN,Physical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Group Key,vwr.com URL,Action Requested,VWR-NA Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,NaN,Size,Pack Type,Search Keywords,NaN,Product Name,Product Text,Key Features/Benefits,NaN,CAS,MDL,EINECS,REACH,Merck Index,UN,Hazard Class,Packing Group,GHS Symbol(s),GHS Signal Word,Density,Boiling Point,Melting Point,Flash Point,Storage Temperature,NaN,Test 1,Result 1,Test 2,Result 2,Test 3,Result 3,Test 4,Result 4,Test 5,Result 5
9,Examples below…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def fillGlobalProductRevisionChemicals(filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept):
    revision = pd.read_excel('forms/GlobalProductRevisionFile_Chemicals (New).xlsx')
    revision.columns = revision.iloc[8]
    new_columns = [i.strip() if type(i) == str else i for i in revision.columns]
    revision.columns = new_columns
    wb = opxl.load_workbook(filename)
    skus = wb.active
    for i in range(2, skus.max_row+1):
        revision.loc[i+11, 'Supplier Part No.'] = str(skus['A'+str(i)].value)

    for i in range(13, len(revision)):
        sku = revision.loc[i, 'Supplier Part No.']
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]

        if not prms_info.empty:
            storage_temp = prms_info['Storage Temp'].values[0]
                
            if storage_temp == 'AM':
                revision.loc[i, 'Storage Temperature'] = '15C to 30C'
            elif storage_temp == 'FR':
                revision.loc[i, 'Storage Temperature'] = '-30C to -2C'
            elif storage_temp == 'RF':
                revision.loc[i, 'Storage Temperature'] = '2C to 8C'
            elif storage_temp == '70' or storage_temp == '80':
                revision.loc[i, 'Storage Temperature'] = '-70C'

        if not product_info_sept.empty:
            keywords = product_info_sept['keywords'].values[0]
            name = product_info_sept['name'].values[0]
            description = product_info_sept['description'].values[0]
            short_description = product_info_sept['short_description'].values[0]
            pack_size_joined = product_info_sept['pack_size_joined'].values[0]
            cas_number = product_info_sept['cas_number'].values[0]
            mdl_number = product_info_sept['mdl_number'].values[0]
            un_number = product_info_sept['un_number'].values[0]
            packing_group = product_info_sept['packing_group'].values[0]
            density = product_info_sept['density'].values[0]
            boiling_point = product_info_sept['boiling_point'].values[0]
            melting_point = product_info_sept['melting_point'].values[0]

            if type(keywords) == str:
                keywords = keywords.replace(', ', ';')
                keywords = keywords.replace(',', ';')
                keywords = keywords.replace(' |', ';')
                keywords = keywords.replace('| ', ';')
                keywords = keywords.replace(' | ', ';')
                keywords = keywords.replace('|', ';')
                revision.loc[i, 'Search Keywords'] = keywords

            if type(name) == str:
                revision.loc[i, 'Product Name'] = tidyDescription(name)
            
            if type(description) == str:
                revision.loc[i, 'Product Text'] = tidyDescription(description)
            
            if type(short_description) == str:
                revision.loc[i, 'Key Features/Benefits'] = tidyDescription(short_description)

            revision.loc[i, 'Supplier Name'] = 'MP Biomedicals'
            revision.loc[i, 'Size'] = pack_size_joined
            revision.loc[i, 'CAS'] = cas_number
            revision.loc[i, 'MDL'] = mdl_number
            revision.loc[i, 'UN'] = un_number
            revision.loc[i, 'Packing Group'] = packing_group
            revision.loc[i, 'Density'] = density
            revision.loc[i, 'Boiling Point'] = boiling_point
            revision.loc[i, 'Melting Point'] = melting_point
    
    revision.columns = revision.columns.fillna('Blank')
                               
    new_revision = opxl.load_workbook('forms/GlobalProductRevisionFile_Chemicals (New).xlsx')
    sheet = new_revision.active
    if sheet.max_row < len(revision):
        sheet.append([''])
        for j in range(len(revision) - sheet.max_row + 4):
            sheet.insert_rows(sheet.max_row)
    i = 14
    for row in sheet.iter_rows(min_row=16):
        if i < (len(revision)):
            for j in range(len(revision.columns)):
                row[j].value = revision.iloc[i, j]
        else:
            break
        i += 1
            
    new_revision.save('outputs/revision_outputs/GlobalProductRevisionChemicals_output.xlsx')

In [20]:
revision_chemicals = fillGlobalProductRevisionChemicals('input_sheets/VWR testing.xlsx', database_sheets[0], database_sheets[1], database_sheets[2], database_sheets[3], database_sheets[4], database_sheets[5], database_sheets[6])

In [8]:
revision_chemicals = revision_chemicals.loc[:, revision_chemicals.columns.notnull()]
revision_chemicals

,Group Key,vwr.com URL,Action Requested,VWR-NA Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Size,Pack Type,Search Keywords,Product Name,Product Text,Key Features/Benefits,CAS,MDL,EINECS,REACH,Merck Index,UN,Hazard Class,Packing Group,GHS Symbol(s),GHS Signal Word,Density,Boiling Point,Melting Point,Flash Point,Storage Temperature,Test 1,Result 1,Test 2,Result 2,Test 3,Result 3,Test 4,Result 4,Test 5,Result 5,Supplier Name
0,"-To submit a Revision File, you must alread...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"-On this form, only populate fields that re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-Use Column C to describe the change you ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-If you leave a column empty than any exist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Product Characteristics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Product Specifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Part Numbers and Grouping,NaN,NaN,NaN,NaN,NaN,NaN,Item Descriptions,NaN,NaN,Text Fields,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Add additional columns as needed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Identification,NaN,NaN,NaN,NaN,Hazard,NaN,NaN,NaN,NaN,Physical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Group Key,vwr.com URL,Action Requested,VWR-NA Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Size,Pack Type,Search Keywords,Product Name,Product Text,Key Features/Benefits,CAS,MDL,EINECS,REACH,Merck Index,UN,Hazard Class,Packing Group,GHS Symbol(s),GHS Signal Word,Density,Boiling Point,Melting Point,Flash Point,Storage Temperature,Test 1,Result 1,Test 2,Result 2,Test 3,Result 3,Test 4,Result 4,Test 5,Result 5,NaN
9,Examples below…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
